## Multiclass Cyberbullying dectection Notebook

### installing necessary libraries

In [1]:
!pip install emoji
!pip install contractions
!pip install langdetect
!pip install mlflow
!pip install pyngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.3/356.3 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.5.1-py2.py3-none-any.whl size=351210 sha256=5da415e1aefe5866ebd736b1382db408756e618b193097acb9af8b4ef26944a0
  Stored in directory: /root/.cache/pip/wheels/51/92/44/e2ef13f803aa08711819357e6de0c5fe67b874671141413565
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wh

### Importing libraries

In [2]:
import pandas as pd
import numpy as np
import re
import emoji
import matplotlib.pyplot as plt
import unicodedata
import contractions
from langdetect import detect
import nltk
import seaborn as sns
import mlflow
import mlflow.sklearn
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.preprocessing import LabelEncoder
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import SMOTE
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import learning_curve,ShuffleSplit
from sklearn.metrics import roc_auc_score,confusion_matrix,roc_curve,auc,mean_squared_error,mean_absolute_error,r2_score,f1_score



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### MLflow

In [3]:
mlflow.set_experiment('ML-project1')

2023/06/26 08:41:25 INFO mlflow.tracking.fluent: Experiment with name 'ML-project1' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/779627010302941359', creation_time=1687768885710, experiment_id='779627010302941359', last_update_time=1687768885710, lifecycle_stage='active', name='ML-project1', tags={}>

### EDA

#### importing the dataset

In [4]:
df = pd.read_csv("***")
df.head()

FileNotFoundError: ignored

In [ ]:
original_df = df.copy()
original_df.head()

#### removing the duplicated data

In [ ]:
print('length before dropping duplicates',len(df))
df = df.drop_duplicates()
df = df.replace('',np.nan)
df.dropna(inplace=True)

print('Length after dropping duplicates: ',len(df))
print('Length of the original dataframe:',len(original_df))


#### Checking for imbalance

In [ ]:
df['cyberbullying_type'].value_counts()

#### Visualizing the Data

In [ ]:
df.groupby('cyberbullying_type').size().plot(kind='pie',autopct='%1.1f%%');

### Text Preprocessing

#### Text cleaning functions

In [ ]:
def emoji_to_text(text):
    if type(text) != float:
        return emoji.demojize(text)
    else:
        return text
print(emoji_to_text('RT @mykitchenrules: Nawwww 😭😭😭 #MKR'))

In [ ]:
def remove_underscore_dash(text):
  return re.sub(r"(-|_)"," ",text)

print(remove_underscore_dash("RT @mykitchenrules: Nawwww :loudly_crying_face::loudly_crying_face::loudly_crying_face: #MKR"))

In [ ]:
def remove_hashtags(text):
    return text.replace('#','')
print(remove_hashtags('#simpleasthat'))

In [ ]:
def remove_mention(text):
  return re.sub(r"@[^\s]+","",text)

print(remove_mention('@girlziplocked jfc.'))

In [ ]:
def remove_link(text):
  return re.sub(r"http[^s]+","",text)
print(remove_link('@Vrais66 And these. http://t.co/V3cf0aGlcp'))

In [ ]:
def remove_repeated_chars(text):
  return re.sub(r"(.)\1\1+",r"\1",text)
print(remove_repeated_chars('“@PublimetroMX: Ninel Conde, víctima del bullying en Twitter, cambia su cuenta http://t.co/pFE73I7”/// aaaaaaaaaajajajajajajajahahahajahaja'))

In [ ]:
def remove_numbers(text):
  return re.sub(r'[0-9]','',text)

print(remove_numbers('@scholarshipscom My 2013 education resolution is to go to school for pharmacy and be successful.'))

In [ ]:
def remove_punctuation(text):
  if not (re.match(r'#(\w+)', text)):
    text = re.sub(r'[^\w\s]',' ',text)
  return text


print(remove_punctuation('My fav #MKR teams so far are definitely: Eva &amp; Debra Rose &amp; Josh Annie &amp; Loyd Robert &amp; Lynzy Sherie &amp; Emilie Jaz &amp; Shaz (not in order)'))

In [ ]:
def remove_non_english_char(text):
  return  "".join(([idx for idx in text if not re.findall("[^\u0000-\u05C0\u2100-\u214F]+", idx)]))

print(remove_non_english_char("kdls # ! 0L نينصه"))

In [ ]:
def remove_whitespace(text):

    text = re.sub(r'^\s+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text).strip('\'')

    return text

print(remove_whitespace('@dsmyxe @PolitiBunny the end        result is.         the same.'))



In [ ]:
def lower_text(text):
  return text.lower()

print(lower_text('Dont like liver #mkr #mkr2015'))

In [ ]:
def remove_accents(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    text = nfkd_form.encode('ASCII', 'ignore').decode('utf-8')
    return text
print(remove_accents('Read my response to "Já sofreram bullying?": http://t.co/sbEIykB'))

In [ ]:
def remove_contractions(text):
  expanded_words = []
  for word in text.split():
      expanded_words.append(contractions.fix(word))

  return ' '.join(expanded_words)

print(remove_contractions("isn't chilling"))

In [ ]:
def remove_non_english_text(text):
  print(detect(text))

remove_non_english_text("Frio é bullying contra os solteiros..")

In [ ]:

def remove_duplicated_emojies(text):
  return re.sub(r"([\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U00002600-\U000027BF])\1+",r'\1',text)

remove_duplicated_emojies("Twitter makes me laugh....Better than the series ;)))))) Very funny 👏👏👏👏👏  #MKR")




In [ ]:
def remove_duplicated_words(text):
  return re.sub(r"\\b(\\w+)(?:\\W+\\1\\b)+",'',text)

In [ ]:

def remove_stopwords(text,flag):
  if flag:
    stop_words = set(stopwords.words('english'))
  else:
    stop_words = set(stopwords.words('english'))
    stop_words.update(['RT','as','go','rt','would','know','one','u','amp','mkr'],)
  tokenizer = TweetTokenizer()
  tokenized = tokenizer.tokenize(text)
  removed_sentence = [word for word in tokenized if not word.lower() in stop_words]
  return ' '.join(removed_sentence)

remove_stopwords('in other words mkr rt RT katandandre your food was crapi..',flag = False)


In [ ]:
def stemmer(text):
   tokenizer = TweetTokenizer()
   tokenized = tokenizer.tokenize(text)
   stemmer = PorterStemmer()
   s = []
   for token in tokenized:
      token = stemmer.stem(token)
      s.append(token)
   return ' '.join(s)

stemmer('words katandandre food crapilicious mkr')


In [ ]:
def lemmatizer(text):
  tokenizer = TweetTokenizer()
  tokenized = tokenizer.tokenize(text)
  lemmatizerr = WordNetLemmatizer()
  s = []
  for token in tokenized:
    token = lemmatizerr.lemmatize(token)
    s.append(token)
  return ' '.join(s)


In [ ]:
def preprocess_text(text,flag):

  """
  Process the text and return it to a clean one.
  Input:
        text: a string containing a text.
  Output:
        cleaned_text : text after applying all the cleaning functions.
  """
  text = remove_mention(text)
  text = remove_link(text)
  text = remove_hashtags(text)
  text = remove_duplicated_emojies(text)
  text = emoji_to_text(text)
  text = remove_repeated_chars(text)
  text = remove_numbers(text)
  text = remove_underscore_dash(text)
  text = remove_contractions(text)
  text = remove_non_english_char(text)
  text = remove_punctuation(text)
  text = remove_accents(text)
  text = lower_text(text)
  text = remove_duplicated_words(text)
  text = remove_whitespace(text)
  text = remove_stopwords(text,flag)
  text = lemmatizer(text)


  return text

preprocess_text("Twitter make me laugh....Better than the series ;)))))) Very funny 👏👏👏👏👏  #MKR",False)

#### applying the above functions on the data

In [ ]:
df['processed_text'] = df['tweet_text'].apply(lambda text: preprocess_text(text,flag=True))
df.head()

#### checking again for Nan/empty/duplicated values & imbalaced classes

In [ ]:
df['processed_text'].isna().sum()

In [ ]:
if df['processed_text'].str.strip().empty:
  print('there are empty values')

In [ ]:
df["processed_text"].duplicated().sum()
df.drop_duplicates("processed_text", inplace=True)

In [ ]:
df.cyberbullying_type.value_counts()

removing the other_cyberbullying type class

In [ ]:
df.drop(df[df['cyberbullying_type'] == 'other_cyberbullying'].index, inplace = True)

checking again for class balance

In [ ]:
df.cyberbullying_type.value_counts()

#### Cleaned Data visualization

In [ ]:
class_counts = df['cyberbullying_type'].value_counts()
class_counts.plot(kind='bar')
plt.title('Class Distribution of Cyberbullying Types')
plt.xlabel('Labels')
plt.ylabel('Number of Tweets')
plt.show()

In [ ]:
text_len = []
for text in df.processed_text:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

In [ ]:
df['text_len'] = text_len

In [ ]:
plt.figure(figsize=(7,5))
ax = sns.countplot(x='text_len', data=df[df['text_len']<10], palette='mako')
plt.title('Count of tweets with less than 10 words', fontsize=20)
plt.yticks([])
ax.bar_label(ax.containers[0])
plt.ylabel('count')
plt.xlabel('')
plt.show()



In [ ]:
df.sort_values(by=['text_len'], ascending=False)


#### Removing ouliers
 removing words less that 4 word and more than 100 as they can be outliers

In [ ]:
df = df[df['text_len'] > 3]
df = df[df['text_len'] < 100]

In [ ]:
df

#### data visualization using word cloud

In [ ]:
print("Gender")
text = " ".join(review for review in df[df.cyberbullying_type=='gender'].tweet_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

print("Ethnicity")
text = " ".join(review for review in df[df.cyberbullying_type=='ethnicity'].tweet_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

print("Religion")
text = " ".join(review for review in df[df.cyberbullying_type=='religion'].tweet_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

print("Age")
text = " ".join(review for review in df[df.cyberbullying_type=='age'].tweet_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

print("Not Cyberbullying")
text = " ".join(review for review in df[df.cyberbullying_type=='not_cyberbullying'].tweet_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()


**checking the top 50 unique words as some of them could be outliers and we can remove them from out data**

In [ ]:
for bully in df['cyberbullying_type'].unique():
    top_50_words=df.processed_text[df['cyberbullying_type']==bully].str.split(expand=True).stack().value_counts()[:51]
top_50_words

**Removing the words that are outlieres by adding them to the stopwords list**

In [ ]:
df['processed_text'] = df['tweet_text'].apply(lambda text: preprocess_text(text,flag=False))
df.head()

#### visualization using word cloud after removing outliers

In [ ]:
print("Gender")
text = " ".join(review for review in df[df.cyberbullying_type=='gender'].processed_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

print("Ethnicity")
text = " ".join(review for review in df[df.cyberbullying_type=='ethnicity'].processed_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

print("Religion")
text = " ".join(review for review in df[df.cyberbullying_type=='religion'].processed_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

print("Age")
text = " ".join(review for review in df[df.cyberbullying_type=='age'].processed_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

print("Not Cyberbullying")
text = " ".join(review for review in df[df.cyberbullying_type=='not_cyberbullying'].processed_text.astype(str))
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()


### Label Encoding

In [ ]:
#### encoding cyberbullying types
encoder=LabelEncoder()
df['cyberbullying_type_encoded']=encoder.fit_transform(df['cyberbullying_type'])
df


### Feature Engineering

**Adding new columns by making some calculations on the tweets to check we can have relations between them**

In [ ]:
def count_chars(text):
  return len(text)


In [ ]:
def count_words(text):
  return len(text.split())



In [ ]:
def count_capital_chars(text):
  count = 0
  for i in text:
    if i.isupper():
      count+=1
  return count

count_capital_chars("Hello MY Dear")

In [ ]:
def count_capital_words(text):
  return sum(map(str.isupper,text.split()))

count_capital_words("Hello MY Dear")

In [ ]:
def count_sent(text):
    return len(nltk.sent_tokenize(text))

count_sent('hello, rami. hello.')

In [ ]:
def count_unique_words(text):
  unique_words = set(word.lower() for word in text.split())
  return len(unique_words)
count_unique_words('hello rami Hello')

In [ ]:
def count_stopwords(text):
  stop_words = set(stopwords.words('english'))
  tokenizer = TweetTokenizer()
  tokenized = tokenizer.tokenize(text)
  stopwords_ = [w for w in tokenized if w in stop_words]
  return len(stopwords_)

count_stopwords("Hello is are i am this that")

creating the new columns

In [ ]:
df['char_count'] = df['tweet_text'].apply(lambda text: count_chars(text))
df['word_count'] = df['tweet_text'].apply(lambda text: count_words(text))
df['sent_count'] = df['tweet_text'].apply(lambda text: count_sent(text))
df['capital_char_count']= df['tweet_text'].apply(lambda text: count_capital_chars(text))
df['capital_word_count'] = df['tweet_text'].apply(lambda text: count_capital_words(text))
df['stopword_count'] = df['tweet_text'].apply(lambda text: count_stopwords(text))
df['unique_word_count'] = df['tweet_text'].apply(lambda text: count_unique_words(text))
df['avg_wordlength'] = df['char_count'] / df ['word_count']
df['avg_senlength'] = df['word_count'] / df['sent_count']
df['unique_vs_words'] = df['unique_word_count'] / df['word_count']
df['stopwords_vs_words'] = df['stopword_count'] / df['word_count']

In [ ]:
df.head()

### Feature extraction




#### tf-idf & BOW vectorization

In [ ]:
tfidf = TfidfTransformer()
clf = CountVectorizer()

X1_cv = clf.fit_transform(df['processed_text'])

tf1_transformer = TfidfTransformer(use_idf=True).fit(X1_cv)
X1_tf = tf1_transformer.transform(X1_cv)
vocabulary = clf.vocabulary_



In [ ]:
X1_tf

In [ ]:
y = df['cyberbullying_type_encoded']
X1_train, X1_test, y_train, y_test = train_test_split(X1_tf, y,test_size= 0.2,random_state=42)

**Checking for class imbalance and as we can see that the clases after splitted are imbalanced**

In [ ]:
y_train.value_counts()

### Balancing Dataset

In [ ]:
vc = y_train.value_counts()
while (vc[0] != vc[4]) or (vc[0] != vc[2]) or (vc[0] != vc[3]) or (vc[0] != vc[1]):
  smote = SMOTE(sampling_strategy='minority')
  X1_train, y_train = smote.fit_resample(X1_train,y_train)
  vc = y_train.value_counts()

y_train.value_counts()

In [ ]:
sentiments = ["religion","age","gender","ethnicity","not bullying"]

### ML Algorithms

#### Naive bais

In [ ]:
def Naive_bias(X_train, y_train, X_test, y_test):
    with mlflow.start_run():
        nb_clf = MultinomialNB()
        nb_clf.fit(X_train, y_train)
        nb_pred = nb_clf.predict(X_test)
        nb_pred_train = nb_clf.predict(X_train)

        f1_score_test = f1_score(y_test, nb_pred, average='weighted')
        f1_score_train = f1_score(y_train, nb_pred_train, average='weighted')

        mlflow.log_metric("f1_score_test", f1_score_test)
        mlflow.log_metric("f1_score_train", f1_score_train)
        mlflow.sklearn.log_model(nb_clf, "naive bayes classifier")

        print('accuracy score (train) :', accuracy_score(y_train, nb_pred_train))
        print('accuracy score  (test) :', accuracy_score(y_test, nb_pred))
        print('F1 score (train) :', f1_score_train)
        print('F1 score (test) :', f1_score_test)

        return classification_report(y_test, nb_pred, target_names=sentiments)


#### Random Forest

In [ ]:
def Random_forest(X_train, y_train, X_test,y_test):
   with mlflow.start_run():
        rf_clf = RandomForestClassifier()
        rf_clf.fit(X_train, y_train)
        rf_pred = rf_clf.predict(X_test)
        rf_pred_train = rf_clf.predict(X_train)

        f1_score_test = f1_score(y_test, rf_pred, average='weighted')
        f1_score_train = f1_score(y_train, rf_pred_train, average='weighted')

        mlflow.log_metric("f1_score_test", f1_score_test)
        mlflow.log_metric("f1_score_train", f1_score_train)
        mlflow.sklearn.log_model(rf_clf, "Random Forest classifier")

        print('accuracy score (train) :', accuracy_score(y_train, rf_pred_train))
        print('accuracy score (test) :', accuracy_score(y_test, rf_pred))
        print('F1 score (train) :', f1_score_train)
        print('F1 score (test) :', f1_score_test)

        return classification_report(y_test, rf_pred, target_names=sentiments)


#### Decision Tree

In [ ]:
def Decision_tree(X_train, y_train, X_test,y_test):
  with mlflow.start_run():
        dt_clf = DecisionTreeClassifier(criterion="entropy")
        dt_clf.fit(X_train, y_train)
        dt_pred = dt_clf.predict(X_test)
        dt_pred_train = dt_clf.predict(X_train)

        f1_score_test = f1_score(y_test, dt_pred, average='weighted')
        f1_score_train = f1_score(y_train, dt_pred_train, average='weighted')

        mlflow.log_metric("f1_score_test", f1_score_test)
        mlflow.log_metric("f1_score_train", f1_score_train)
        mlflow.sklearn.log_model(dt_clf, "decision tree classifier")

        print('accuracy score (train) :', accuracy_score(y_train, dt_pred_train))
        print('accuracy score (test) :', accuracy_score(y_test, dt_pred))
        print('F1 score (train) :', f1_score_train)
        print('F1 score (test) :', f1_score_test)

        return classification_report(y_test, dt_pred, target_names=sentiments)


#### Support Vector Machine

In [ ]:
def Svc(X_train, y_train, X_test, y_test):
  with mlflow.start_run():
        svc_clf = svm.SVC(kernel='linear', C=1, gamma=0)
        svc_clf.fit(X_train, y_train)
        svc_pred = svc_clf.predict(X_test)
        svc_pred_train = svc_clf.predict(X_train)

        f1_score_test = f1_score(y_test, svc_pred, average='weighted')
        f1_score_train = f1_score(y_train, svc_pred_train, average='weighted')

        mlflow.log_metric("f1_score_test", f1_score_test)
        mlflow.log_metric("f1_score_train", f1_score_train)
        mlflow.sklearn.log_model(svc_clf, "svm classifier")

        print('accuracy score (train) :', accuracy_score(y_train, svc_pred_train))
        print('accuracy score (test) :', accuracy_score(y_test, svc_pred))
        print('F1 score (train) :', f1_score_train)
        print('F1 score (test) :', f1_score_test)

        return classification_report(y_test, svc_pred, target_names=sentiments)


#### Run algorithm

In [ ]:
def run_algorithm(X_train, y_train, X_test,y_test):
  print('Naive Bayes:')
  print("Classification Report for Naive Bias :",Naive_bias(X_train,y_train,X_test,y_test))
  print('______________________________________________________________________')
  print('Random Forest :')
  print("Classification Report for Random Forest :",Random_forest(X_train,y_train,X_test,y_test))
  print('______________________________________________________________________')
  print('Decision Tree :')
  print("Classification Reportj for Decision Tree:",Decision_tree(X_train, y_train, X_test, y_test))
  print('______________________________________________________________________')
  print('Support Vector Machines: ')
  print('Classification Report for Support Vector Machines',Svc(X_train, y_train, X_test, y_test))


In [ ]:
run_algorithm(X1_train, y_train, X1_test,y_test)

### Hyperparameter Tuning

#### Support Vector Machines

In [ ]:
def svc_gridSCV(X_train,y_train,X_test,y_test):
  param_grid = {
    'C': uniform(loc=0, scale=10),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'degree': [1,2, 3,] }
  svc_clf = svm.SVC()
  random_search = RandomizedSearchCV(svc_clf,param_distributions = param_grid, n_iter=10 , cv=5,scoring='accuracy',random_state=42)
  random_search.fit(X_train,y_train)
  svc_rs_pred = random_search.predict(X_test)
  svc_rs_pred_train = random_search.predict(X_train)
  print('accuracy score (train) :',accuracy_score(y_train,svc_rs_pred_train))
  print('accuracy score (test) :', accuracy_score(y_test, svc_rs_pred))

  print("Best Parameters: ", random_search.best_params_)
  print("Best Score: ", random_search.best_score_)

  return classification_report(y_test,svc_rs_pred,target_names = sentiments)


#svc_gridSCV(X1_train, y_train, X1_test,y_test)


# accuracy score (train) : 0.9996851881001102
# accuracy score (test) : 0.9316135966680102
# Best Parameters:  {'C': 5.247564316322379, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
# Best Score:  0.9389894538013538
#               precision    recall  f1-score   support

#     religion       0.96      0.97      0.96      1537
#          age       0.98      0.98      0.98      1571
#       gender       0.95      0.87      0.91      1462
#    ethnicity       0.79      0.87      0.83      1320
# not bullying       0.96      0.95      0.96      1553

#     accuracy                           0.93      7443
#    macro avg       0.93      0.93      0.93      7443
# weighted avg       0.93      0.93      0.93      7443



#### Random Forest

In [ ]:





def rf_gridSCV(X_train,y_train,X_test,y_test):

  param_grid = {
    'n_estimators': [10, 50, 100, 200, 500],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None] }

  rfـclf = RandomForestClassifier()
  random_search = RandomizedSearchCV(rfـclf,param_distributions = param_grid, n_iter=10 , cv=5,random_state=42)
  random_search.fit(X_train,y_train)
  rf_rs_pred = random_search.predict(X_test)
  rf_rs_pred_train = random_search.predict(X_train)
  print('accuracy score (train) :',accuracy_score(y_train,rf_rs_pred_train))
  print('accuracy score (test) :', accuracy_score(y_test, rf_rs_pred))

  print("Best Parameters: ", random_search.best_params_)
  print("Best Score: ", random_search.best_score_)

  return classification_report(y_test,rf_rs_pred,target_names = sentiments)

#rf_gridSCV(X1_train, y_train, X1_test,y_test)


# accuracy score (train) : 0.9272469699354635
# accuracy score (test) : 0.9181781539701733
# Best Parameters:  {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 20}
# Best Score:  0.9173933574689123

#               precision    recall  f1-score   support

#     religion       0.99      0.96      0.98      1537
#          age       0.98      0.96      0.97      1571
#       gender       0.98      0.80      0.88      1462
#    ethnicity       0.71      0.95      0.81      1320
# not bullying       0.97      0.91      0.94      1553

#     accuracy                           0.92      7443
#    macro avg       0.93      0.92      0.92      7443
# weighted avg       0.93      0.92      0.92      7443


#### Decision Tree

In [ ]:
def dt_gridSCV(X_train,y_train,X_test,y_test):

  param_grid = {
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
  }
  dt_clf = DecisionTreeClassifier()
  random_search = RandomizedSearchCV(dt_clf,param_distributions = param_grid, n_iter=10 , cv=5,random_state=42)
  random_search.fit(X_train,y_train)
  dt_rs_pred = random_search.predict(X_test)
  dt_rs_pred_train = random_search.predict(X_train)
  print('accuracy score (train) :',accuracy_score(y_train,dt_rs_pred_train))
  print('accuracy score (test) :', accuracy_score(y_test, dt_rs_pred))

  print("Best Parameters: ", random_search.best_params_)
  print("Best Score: ", random_search.best_score_)

  return classification_report(y_test,dt_rs_pred,target_names = sentiments)


#dt_gridSCV(X1_train, y_train, X1_test,y_test)






# accuracy score (train) : 0.85997166692901
# accuracy score (test) : 0.847776434233508
# Best Parameters:  {'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 10}
# Best Score:  0.854525421060916


#               precision    recall  f1-score   support

#     religion       0.99      0.93      0.96      1537
#          age       0.98      0.86      0.92      1571
#       gender       0.99      0.64      0.77      1462
#    ethnicity       0.55      0.97      0.70      1320
# not bullying       0.97      0.85      0.90      1553

#     accuracy                           0.85      7443
#    macro avg       0.90      0.85      0.85      7443
# weighted avg       0.91      0.85      0.86      7443




###Ensemble learning

#### Soft Voting

In [ ]:
def soft_voting(X_train, y_train, X_test, y_test):
  with mlflow.start_run():
    svm_clf = svm.SVC(C= 5.247564316322379, degree= 1, gamma= 'scale', kernel= 'rbf', probability=True)
    rf_clf = RandomForestClassifier(n_estimators= 50, min_samples_split= 10, min_samples_leaf= 2, max_features= None, max_depth= 20, bootstrap=True , oob_score=True)
    dt_clf = DecisionTreeClassifier(min_samples_split= 10, min_samples_leaf= 2, max_features= None, max_depth= 10)

    voting_clf = VotingClassifier(
        estimators=[('rf', rf_clf), ('dt', dt_clf)],
        voting='soft',
        flatten_transform=True,
        n_jobs=-1
    )

    title = "Learning Curves (Soft Voting Classifier)"
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    plot_learning_curve(voting_clf, title, X_train, y_train, ylim=(0.7, 1.01), cv=cv, n_jobs=-1)

    voting_clf.fit(X_train, y_train)

    voting_clf_pred = voting_clf.predict(X_test)
    voting_clf_train = voting_clf.predict(X_train)
    rmse,mae,r2 = eval(y_test,voting_clf_pred)
    f1_score_test = f1_score(y_test, rf_pred, average='weighted')
    f1_score_train = f1_score(y_train, rf_pred_train, average='weighted')

    mlflow.log_metric("f1_score_test", f1_score_test)
    mlflow.log_metric("f1_score_train", f1_score_train)
    mlflow.sklearn.log_model(voting_clf,"softvoting classifier")



    print('accuracy score (train) :',accuracy_score(y_train,voting_clf_train))
    print('accuracy score (test) :', accuracy_score(y_test, voting_clf_pred))
    print(classification_report(y_test,voting_clf_pred,target_names=sentiments))

    return voting_clf



### Model Evaluation

#### Confusion matrix & Learning Curve

In [ ]:


def plot_learning_curve_with_auc_and_confusion_matrix(estimator, title, X, y, ylim=None, cv=None, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure(figsize=(12, 4))

    # Fit the estimator
    estimator.fit(X, y)

    # Plot learning curves
    plt.subplot(1, 2, 1)
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='accuracy')
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.legend(loc="best")

    # Plot confusion matrix
    plt.subplot(1, 2, 2)
    y_pred = estimator.predict(X)
    cm = confusion_matrix(y, y_pred)
    cmap = plt.get_cmap('Blues')
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(np.unique(y)))

    plt.xticks(tick_marks, np.unique(y), rotation=45)
    plt.yticks(tick_marks, np.unique(y))

    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()



    return plt



def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    return plot_learning_curve_with_auc_and_confusion_matrix(estimator, title, X, y, ylim, cv, n_jobs, train_sizes)



soft_voting_clf = soft_voting(X1_train, y_train, X1_test, y_test)
plt.show()


#### ROC AUC

In [ ]:
def plot_roc(clf,X_test, y_test):

  pred = clf.predict(X_test)
  pred_proba = clf.predict_proba(X_test)

  fpr = {}
  tpr = {}
  thresh = {}
  roc_auc = {}
  n_class = 5
  color_mat = ['orange','green','blue','red','purple']

  for i in range(n_class):
     fpr[i], tpr[i], _ = roc_curve(y_test, pred_proba[:, i], pos_label=i)
     roc_auc[i] = auc(fpr[i], tpr[i])
     string = 'class' + str(i) + 'vs Rest (AUC = %0.2f)' % roc_auc[i]
     plt.plot(fpr[i], tpr[i], linestyle='--', color=color_mat[i], label=string)

  plt.title('Multiclass ROC curve')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive rate')
  plt.legend(loc='best')
  plt.savefig('Multiclass ROC',dpi=300);

plot_roc(soft_voting_clf,X1_test,y_test)
plt.show()






### Checking on ML Flow

In [ ]:
from pyngrok import ngrok

In [ ]:
! ngrok config add-authtoken #my_auth_token

In [ ]:

# public_url = ngrok.connect(port = '8050')


ngrok.kill()
#2RGUGK8CBB79RAtl8U242lWSv2G_a3vux6PfHfVEhjaFDhjV
NGROK_AUTH_TOKEN = #My_auth_token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="5000",proto="http",bind_tls=True)
print("ML Project Tracking UI",ngrok_tunnel.public_url)

In [ ]:
!mlflow ui

## WEB APP

### Installing the libraries

In [ ]:
!pip install mlflow
!pip install jupyter-dash
!pip install pyngrok
!pip install dash
!pip install dash_core_components
!pip install dash_html_components
!pip install emoji
!pip install contractions
!pip install langdetect

### Importing the libraries

In [ ]:
import mlflow
import sys
import re
import mlflow.sklearn
import numpy as np
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from pyngrok import ngrok
import unicodedata
import contractions
import unicodedata
import contractions
import traceback
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.preprocessing import LabelEncoder


### Loading the model from MLFLOW

In [ ]:
# Load the MLflow model
model_uri = 'runs:/cf57338725f44419a6c5003b2fa19981/softvoting classifier'  # Replace with the URI of your trained MLflow model
model = mlflow.sklearn.load_model(model_uri)
vectorizer = TfidfVectorizer()
#vectorizer.vocabulary_ = vocabulary

# Define the classes for cyberbullying detection
classes = ['age', 'ethnicity','gender','not_cyberbullying','religion']

X_train = df['processed_text']
vectorizer.fit(X_train)

### Preprocessing the Enterd Tweet

In [ ]:

def emoji_to_text(text):
    if type(text) != float:
        return emoji.demojize(text)
    else:
        return text

def remove_underscore_dash(text):
  return re.sub(r"(-|_)"," ",text)

def remove_hashtags(text):
    return text.replace('#','')

def remove_mention(text):
  return re.sub(r"@[^\s]+","",text)

def remove_link(text):
  return re.sub(r"http[^s]+","",text)

def remove_repeated_chars(text):
  return re.sub(r"(.)\1\1+",r"\1",text)

def remove_numbers(text):
  return re.sub(r'[0-9]','',text)

def remove_punctuation(text):
  if not (re.match(r'#(\w+)', text)):
    text = re.sub(r'[^\w\s]',' ',text)
  return text

def remove_non_english_char(text):
  return  "".join(([idx for idx in text if not re.findall("[^\u0000-\u05C0\u2100-\u214F]+", idx)]))


def remove_whitespace(text):

    text = re.sub(r'^\s+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text).strip('\'')

    return text

def lower_text(text):
  return text.lower()

def remove_accents(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    text = nfkd_form.encode('ASCII', 'ignore').decode('utf-8')
    return text

def remove_contractions(text):
  expanded_words = []
  for word in text.split():
      expanded_words.append(contractions.fix(word))

  return ' '.join(expanded_words)

def remove_duplicated_emojies(text):
  return re.sub(r"([\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U00002600-\U000027BF])\1+",r'\1',text)

def remove_duplicated_words(text):
  return re.sub(r"\\b(\\w+)(?:\\W+\\1\\b)+",'',text)


def remove_stopwords(text,flag):
  if flag:
    stop_words = set(stopwords.words('english'))
  else:
    stop_words = set(stopwords.words('english'))
    stop_words.update(['RT','as','go','rt','would','know','one','u','amp','mkr'],)
  tokenizer = TweetTokenizer()
  tokenized = tokenizer.tokenize(text)
  removed_sentence = [word for word in tokenized if not word.lower() in stop_words]
  return ' '.join(removed_sentence)

def stemmer(text):
   tokenizer = TweetTokenizer()
   tokenized = tokenizer.tokenize(text)
   stemmer = PorterStemmer()
   s = []
   for token in tokenized:
      token = stemmer.stem(token)
      s.append(token)
   return ' '.join(s)

def lemmatizer(text):
  tokenizer = TweetTokenizer()
  tokenized = tokenizer.tokenize(text)
  lemmatizerr = WordNetLemmatizer()
  s = []
  for token in tokenized:
    token = lemmatizerr.lemmatize(token)
    s.append(token)
  return ' '.join(s)






def preprocess_text(text,flag):

  """
  Process the text and return it to a clean one.
  Input:
        text: a string containing a text.
  Output:
        cleaned_text : text after applying all the cleaning functions.
  """
  text = remove_mention(text)
  text = remove_link(text)
  text = remove_hashtags(text)
  text = remove_duplicated_emojies(text)
  text = emoji_to_text(text)
  text = remove_repeated_chars(text)
  text = remove_numbers(text)
  text = remove_underscore_dash(text)
  text = remove_contractions(text)
  text = remove_non_english_char(text)
  text = remove_punctuation(text)
  text = remove_accents(text)
  text = lower_text(text)
  text = remove_duplicated_words(text)
  text = remove_whitespace(text)
  text = remove_stopwords(text,flag)
  text = lemmatizer(text)




  return text

### Running the web app

In [ ]:






# Create the Dash app
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Cyberbullying Detection"),
    dcc.Textarea(
        id='input-text',
        placeholder="Enter the text here...",
        style={'width': '100%', 'height': 100}
    ),
    html.Button('Submit', id='submit-button', n_clicks=0),
    html.Div(id='output-prediction')
])

# Define the callback function to handle text classification
@app.callback(
    Output('output-prediction', 'children'),
    [Input('submit-button', 'n_clicks')],
    [dash.dependencies.State('input-text', 'value')]
)


def classify_text(n_clicks, text):
    if n_clicks > 0 and text:
        try:

            preprocessed_text = preprocess_text(text, True)
            vectorized_text = vectorizer.transform([preprocessed_text]).toarray()

            #print("Preprocessed Text:", preprocessed_text)


            # Make predictions on the preprocessed text using the trained model
            predicted_class = model.predict(vectorized_text)[0]
            class_label = classes[predicted_class]

            return f"The text belongs to the class: {class_label}"
        except Exception as e:
            traceback_str = traceback.format_exc()
            print(traceback_str)
            return html.Div([html.Pre(traceback_str, style={'color': 'red'})])
            return "An error occurred during text classification."
    return ''





# Run the app and display the result using ngrok
if __name__ == '__main__':
    app.run_server(mode='external')
    ngrok.kill()
    public_url = ngrok.connect(addr='localhost:8050')

    print("Dash app running at:", public_url)

